In [1]:
import re
import pandas as pd
import datetime
import mysql.connector


In [2]:
conexao = mysql.connector.connect(host='localhost',user='root',password='',database='producao')
cursor = conexao.cursor()

In [3]:
def LerFilename(revista):
    comando = f'select * from rpis_lidas where rpi="{revista}"'
    cursor.execute(comando)
    resultado = cursor.fetchall()
    #print(len(resultado))
    for row in resultado:
        #print(row[0])
        data_revista = row[1]
    df = pd.DataFrame(resultado)
    data = df[1].astype("string")
    str_data_revista = str(data_revista)
    d=datetime.datetime.strptime(str_data_revista, "%Y-%m-%d")
    #print(d.year)
    aux = str(d.strftime("%d%m%Y"))
    filename = f'revistas/Patente_{revista}_{aux}.xml'
    return filename, str_data_revista

revista = 2665
filename, data_revista = LerFilename(revista)
print(filename)
print(data_revista)

revistas/Patente_2665_01022022.xml
2022-02-01


In [4]:
def ConverteNumero(x):
    s = str(x)
    s = s.replace("BR","")
    s = s.replace(" ","")
    pos = s.find('-')
    return s[0:pos]

In [5]:
def ConverteData(data):
    d=datetime.datetime.strptime(data, "%d/%m/%Y")
    return f'{d.year:04}-{d.month:02}-{d.day:02}'

In [6]:
def LerRPI(filename,data_revista):
    file = open(file=filename,mode='r',encoding="utf8")
    file_content_raw=file.read()
    text1=re.compile("<despacho>")
    file_content=text1.split(file_content_raw)
    file.close()
    texto=file_content[0]
    t = re.compile(r'\d{2}/../\d{4}')
    check = t.findall(texto)
    data = ''.join(check)
    data=ConverteData(data)
    if data_revista == data:
        print("Data da RPI "+data_revista+" confere com a data gravada dentro do XML "+data) 
    else:
        print("Data da RPI "+data_revista+" não confere com a data gravada dentro do XML "+data) 
    return file_content

file_content = LerRPI(filename,data_revista)
print("Total registros: "+str(len(file_content)-1))

Data da RPI 2022-02-01 confere com a data gravada dentro do XML 2022-02-01
Total registros: 4189


In [7]:
codigo_tag = re.compile('<codigo>(.*?)</codigo>')
titulo_tag = re.compile('<titulo>(.*?)</titulo>')
comentario_tag = re.compile("<comentario inid=\"co\">(.*?)</comentario>") 
nome_completo_tag = re.compile("<nome-completo>(.*?)</nome-completo>") 
data_deposito_tag = re.compile("<data-deposito inid=\"22\">(.*?)</data-deposito>") 

numero_tag = re.compile("<numero inid=\"21\">(.*?)</numero>") 
numero_a2_tag = re.compile("<numero inid=\"21\" kindcode=\"A2\">(.*?)</numero>") 
numero_e2_tag = re.compile("<numero inid=\"21\" kindcode=\"E2\">(.*?)</numero>") 
numero_u2_tag = re.compile("<numero inid=\"21\" kindcode=\"U2\">(.*?)</numero>") 
numero_a8_tag = re.compile("<numero inid=\"21\" kindcode=\"A8\">(.*?)</numero>") 
numero_u8_tag = re.compile("<numero inid=\"21\" kindcode=\"U8\">(.*?)</numero>") 
numero_21b1_tag = re.compile("<numero inid=\"21\" kindcode=\"B1\">(.*?)</numero>") 
numero_21b8_tag = re.compile("<numero inid=\"21\" kindcode=\"B8\">(.*?)</numero>") 
numero_21y1_tag = re.compile("<numero inid=\"21\" kindcode=\"Y1\">(.*?)</numero>") 
numero_21y8_tag = re.compile("<numero inid=\"21\" kindcode=\"Y8\">(.*?)</numero>") 
numero_21e8_tag = re.compile("<numero inid=\"21\" kindcode=\"E8\">(.*?)</numero>") 

numero_11a8_tag = re.compile("<numero inid=\"11\" kindcode=\"A8\">(.*?)</numero>") 
numero_b1_tag = re.compile("<numero inid=\"11\" kindcode=\"B1\">(.*?)</numero>") 
numero_b8_tag = re.compile("<numero inid=\"11\" kindcode=\"B8\">(.*?)</numero>") 
numero_y1_tag = re.compile("<numero inid=\"11\" kindcode=\"Y1\">(.*?)</numero>") 
numero_y8_tag = re.compile("<numero inid=\"11\" kindcode=\"Y8\">(.*?)</numero>") 
numero_c8_tag = re.compile("<numero inid=\"11\" kindcode=\"C8\">(.*?)</numero>") 
numero_f1_tag = re.compile("<numero inid=\"11\" kindcode=\"F1\">(.*?)</numero>") 
numero_f8_tag = re.compile("<numero inid=\"11\" kindcode=\"F8\">(.*?)</numero>") 
numero_g8_tag = re.compile("<numero inid=\"11\" kindcode=\"G8\">(.*?)</numero>") 

data_fase_nacional_tag = re.compile("<data-fase-nacional inid=\"85\">(.*?)</data-fase-nacional>") 
titulo_pedido_tag = re.compile("<titulo inid=\"54\">(.*?)</titulo>") 
titular_lista_tag = re.compile("<titular-lista>[\s\S]*</titular-lista>")

codigo_list, titulo_list, comentario_list, nome_completo_list, data_deposito_list, numero_list, uf_list, sigla_list, numero_pct_list, data_pct_list, publicacao_internacional_list, numero_ompi_list, data_ompi_list, classe_internacional_list, classe_nacional_list, sigla_prioridade_list, numero_prioridade_list, data_prioridade_list, prioridade_list, prioridade_interna_list, data_fase_nacional_list, titulo_pedido_list, inventor_nome_completo_list, divisao_pedido_list, data_divisao_pedido_list, data_rpi_list, data_concessao_list = ([] for i in range(27))

for line in file_content:
    
    codigo=codigo_tag.findall(line)
    titulo=titulo_tag.findall(line)
    comentario=comentario_tag.findall(line)
    comentario = ', '.join(comentario)

    #to find inventors
    data_deposito=None
    data_fase_nacional=None
    numero=None
    nome_completo=None
    inventor_nome_completo=None
    titulo_pedido=""
    uf=None
    sigla=None
    numero_pct=" "
    data_pct=" "
    numero_ompi=" "
    data_ompi=" "
    classe_internacional=None
    str_classe = ''
    classe_nacional=None
    str_classe_nacional = ''
    sigla_prioridade=" "
    numero_prioridade=" "
    data_prioridade=" "
    str_prioridade=''
    numero_prioridade_interna=" "
    data_prioridade_interna=" "
    str_prioridade_interna=''
    numero_divisao_pedido=" "
    data_divisao_pedido=" "
    data_rpi=" "
    data_concessao=" "
    
    processos=re.findall("<processo-patente>[\s\S]*</processo-patente>",line)
    for processo in processos:
        data_deposito=data_deposito_tag.findall(processo)
        data_deposito=', '.join(data_deposito) # converte lista em string
        numero=numero_tag.findall(processo)
        if len(numero)==0:
            numero=numero_a2_tag.findall(processo)
            if len(numero)==0:
                numero=numero_u2_tag.findall(processo)
                if len(numero)==0:
                    numero=numero_a8_tag.findall(processo)
                    if len(numero)==0:
                        numero=numero_u8_tag.findall(processo)
                        if len(numero)==0:
                            numero=numero_b1_tag.findall(processo)
                            if len(numero)==0:
                                numero=numero_b8_tag.findall(processo)
                                if len(numero)==0:
                                    numero=numero_y1_tag.findall(processo)
                                    if len(numero)==0:
                                        numero=numero_y8_tag.findall(processo)
                                        if len(numero)==0:
                                            numero=numero_e2_tag.findall(processo)
                                            if len(numero)==0:
                                                numero=numero_21b1_tag.findall(processo)
                                                if len(numero)==0:
                                                    numero=numero_21b8_tag.findall(processo)
                                                    if len(numero)==0:
                                                        numero=numero_21y1_tag.findall(processo)
                                                        if len(numero)==0:
                                                            numero=numero_21y8_tag.findall(processo)
                                                            if len(numero)==0:
                                                                numero=numero_c8_tag.findall(processo)
                                                                if len(numero)==0:
                                                                    numero=numero_f1_tag.findall(processo)
                                                                    if len(numero)==0:
                                                                        numero=numero_11a8_tag.findall(processo)
                                                                        if len(numero)==0:
                                                                            numero=numero_21e8_tag.findall(processo)
                                                                            if len(numero)==0:
                                                                                numero=numero_f8_tag.findall(processo)
                                                                                if len(numero)==0:
                                                                                    numero=numero_g8_tag.findall(processo)
        numero = ', '.join(numero)
        data_fase_nacional=data_fase_nacional_tag.findall(processo)
        data_fase_nacional = ', '.join(data_fase_nacional)
        titulo_pedido=titulo_pedido_tag.findall(processo)
        titulo_pedido = ', '.join(titulo_pedido)

        #concessao (16.3)
        concessoes_lista=re.findall("<concessao inid=\"45\">[\s\S]*</concessao>",processo)
        for concessao_lista in concessoes_lista:
            data_concessao=re.findall("<data>(.*?)</data>",concessao_lista)
            
        #publicacao_nacional (16.1)
        publicacoes_nacional_lista=re.findall("<publicacao-nacional inid=\"43\">[\s\S]*</publicacao-nacional>",processo)
        for publicacao_nacional_lista in publicacoes_nacional_lista:
            data_rpi=re.findall("<data-rpi>(.*?)</data-rpi>",publicacao_nacional_lista)
            
        #divisao_pedido
        divisao_pedidos_lista=re.findall("<divisao-pedido inid=\"62\">[\s\S]*</divisao-pedido>",processo)
        for divisao_pedido_lista in divisao_pedidos_lista:
            data_divisao_pedido=re.findall("<data-deposito>(.*?)</data-deposito>",divisao_pedido_lista)
            numero_divisao_pedido=re.findall("<numero>(.*?)</numero>",divisao_pedido_lista)
            
        #titular_lista
        titulares_lista=re.findall("<titular-lista>[\s\S]*</titular-lista>",processo)
        for titular_lista in titulares_lista:
            titulares=re.findall("<titular sequencia=\"1\" inid=\"71\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)
                        
            titulares=re.findall("<titular sequencia=\"1\" inid=\"73\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)

        #inventor_lista
        inventores_lista=re.findall("<inventor-lista>[\s\S]*</inventor-lista>",processo)
        for inventor_lista in inventores_lista:
            #print(inventor_lista)
            inventores=re.findall("<inventor sequencia=\"1\" inid=\"72\">[\s\S]*</inventor>",inventor_lista)
            for inventor in inventores:
                inventor_nome_completo=nome_completo_tag.findall(inventor)
                inventor_nome_completo = '; '.join(inventor_nome_completo)
                        
        #pedido_internacional
        pedidos_internacional_lista=re.findall("<pedido-internacional inid=\"86\">[\s\S]*</pedido-internacional>",processo)
        for pedido_internacional_lista in pedidos_internacional_lista:
            #print(pedido_internacional_lista)
            numeros_pct=re.findall("<numero-pct>[\s\S]*</numero-pct>",pedido_internacional_lista)
            for numero_pct_lista in numeros_pct:
                numero_pct=re.findall("<numero-pct>(.*?)</numero-pct>",numero_pct_lista)
            datas_pct=re.findall("<data-pct>[\s\S]*</data-pct>",pedido_internacional_lista)
            for data_pct_lista in datas_pct:
                data_pct=re.findall("<data-pct>(.*?)</data-pct>",data_pct_lista)

        #publicacao_internacional
        publicacoes_internacional_lista=re.findall("<publicacao-internacional inid=\"87\">[\s\S]*</publicacao-internacional>",processo)
        for publicacao_internacional_lista in publicacoes_internacional_lista:
            numeros_ompi=re.findall("<numero-ompi>[\s\S]*</numero-ompi>",publicacao_internacional_lista)
            for numero_ompi_lista in numeros_ompi:
                numero_ompi=re.findall("<numero-ompi>(.*?)</numero-ompi>",numero_ompi_lista)
            datas_ompi=re.findall("<data-ompi>[\s\S]*</data-ompi>",publicacao_internacional_lista)
            for data_ompi_lista in datas_ompi:
                data_ompi=re.findall("<data-ompi>(.*?)</data-ompi>",data_ompi_lista)

        #classificacao_internacional
        classificacoes_internacional_lista=re.findall("<classificacao-internacional-lista>[\s\S]*</classificacao-internacional-lista>",processo)
        for classificacao_internacional_lista in classificacoes_internacional_lista:
            for i in range(1,20):
                #print(f"<classificacao-internacional inid=\"51\" sequencia=\"{i}\" ano=\"2006.01\" >[\s\S]*</classificacao-internacional>")
                classificacoes_internacional=re.findall(f"<classificacao-internacional inid=\"51\" sequencia=\"{i}\" ano=\"2006.01\">[\s\S]*</classificacao-internacional>",classificacao_internacional_lista)
                for classificacao_internacional in classificacoes_internacional:
                    classe_internacional=re.findall(f"<classificacao-internacional inid=\"51\" sequencia=\"{i}\" ano=\"2006.01\">(.*?)</classificacao-internacional>",classificacao_internacional)
                    if i==1:
                        str_classe = ', '.join(classe_internacional) # converte lista em string
                    else:
                        str_classe = str_classe+'; '+', '.join(classe_internacional) # converte lista em string
                    #print(', '.join(numero)+' '+str_classe)

        #classificacao_nacional
        classificacoes_nacional_lista=re.findall("<classificacao-nacional-lista>[\s\S]*</classificacao-nacional-lista>",processo)
        for classificacao_nacional_lista in classificacoes_nacional_lista:
            for i in range(1,20):
                #print(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\" >[\s\S]*</classificacao-nacional>")
                classificacoes_nacional=re.findall(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\">[\s\S]*</classificacao-nacional>",classificacao_nacional_lista)
                for classificacao_nacional in classificacoes_nacional:
                    classe_nacional=re.findall(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\">(.*?)</classificacao-nacional>",classificacao_nacional)
                    if i==1:
                        str_classe_nacional = ', '.join(classe_nacional) # converte lista em string
                    else:
                        str_classe_nacional = str_classe_nacional+'; '+', '.join(classe_nacional) # converte lista em string
                    #print(', '.join(numero)+' '+str_classe)
                    
        #prioridade_unionista
        prioridades_unionista_lista=re.findall("<prioridade-unionista-lista>[\s\S]*</prioridade-unionista-lista>",processo)
        for prioridade_unionista_lista in prioridades_unionista_lista:
            for i in range(1,5):
                prioridades_unionista=re.findall(f"<prioridade-unionista sequencia=\"{i}\" inid=\"30\">[\s\S]*</prioridade-unionista>",prioridade_unionista_lista)
                for prioridade_unionista in prioridades_unionista:
                    #print(prioridade_unionista)
                    sigla_prioridade=re.findall("<sigla-pais inid=\"33\">(.*?)</sigla-pais>",prioridade_unionista)
                    numero_prioridade=re.findall("<numero-prioridade inid=\"32\">(.*?)</numero-prioridade>",prioridade_unionista)
                    data_prioridade=re.findall("<data-prioridade inid=\"31\">(.*?)</data-prioridade>",prioridade_unionista)
                    sigla_prioridade_str=sigla_prioridade[0]
                    numero_prioridade_str=numero_prioridade[0]
                    data_prioridade_str=data_prioridade[0]
                    if i==1:
                        str_prioridade = sigla_prioridade_str+numero_prioridade_str+' '+data_prioridade_str # converte lista em string
                    else:
                        str_prioridade = str_prioridade+'; '+sigla_prioridade_str+numero_prioridade_str+' '+data_prioridade_str

                    numero_str = ', '.join(numero)
                    if numero_str=='BR 11 2022 020312-8':
                        print(f"<prioridade-unionista sequencia=\"{i}\" inid=\"30\">[\s\S]*</prioridade-unionista>")
                        print(sigla_prioridade)
                        print(numero_prioridade)
                        print(data_prioridade)
                        print(str_prioridade)
 
        #prioridade_interna
        prioridades_interna_lista=re.findall("<prioridade-interna-lista>[\s\S]*</prioridade-interna-lista>",processo)
        for prioridade_interna_lista in prioridades_interna_lista:
            for i in range(1,5):
                prioridades_interna=re.findall(f"<prioridade-interna sequencia=\"{i}\" inid=\"66\">[\s\S]*</prioridade-interna>",prioridade_interna_lista)
                for prioridade_interna in prioridades_interna:
                    numero_prioridade_interna=re.findall("<numero-prioridade>(.*?)</numero-prioridade>",prioridade_interna)
                    data_prioridade_interna=re.findall("<data-prioridade>(.*?)</data-prioridade>",prioridade_interna)
                    numero_prioridade_interna_str=numero_prioridade_interna[0]
                    data_prioridade_interna_str=data_prioridade_interna[0]
                    if i==1:
                        str_prioridade_interna = numero_prioridade_interna_str+' '+data_prioridade_interna_str # converte lista em string
                    else:
                        str_prioridade_interna = str_prioridade_interna+'; '+numero_prioridade_interna_str+' '+data_prioridade_interna_str

                
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(codigo)!=0:                             
        codigo_list.append(codigo[0])
        titulo_list.append(titulo[0])
        comentario_list.append(comentario)
        data_deposito_list.append(data_deposito)
        numero_list.append(numero)
        data_fase_nacional_list.append(data_fase_nacional)
        titulo_pedido_list.append(titulo_pedido)
        nome_completo_list.append(nome_completo)
        inventor_nome_completo_list.append(inventor_nome_completo)
        uf_list.append(uf)
        sigla_list.append(sigla)
        numero_pct_list.append(numero_pct[0])
        data_pct_list.append(data_pct[0])
        numero_ompi_list.append(numero_ompi[0])
        data_ompi_list.append(data_ompi[0])
        classe_internacional_list.append(str_classe)
        classe_nacional_list.append(str_classe_nacional)
        prioridade_list.append(str_prioridade)
        sigla_prioridade_list.append(sigla_prioridade[0])
        numero_prioridade_list.append(numero_prioridade[0])
        data_prioridade_list.append(data_prioridade[0])
        prioridade_interna_list.append(str_prioridade_interna)
        divisao_pedido_list.append(numero_divisao_pedido[0])
        data_divisao_pedido_list.append(data_divisao_pedido[0])
        data_rpi_list.append(data_rpi[0])
        data_concessao_list.append(data_concessao[0])
        


In [8]:
data_frame = pd.DataFrame(
    {'despacho': codigo_list,
     'numero': numero_list,
     'data_deposito': data_deposito_list,
     'data_fase_nacional': data_fase_nacional_list,
     'data_publicacao_nacional': data_rpi_list,
     'data_concessao': data_concessao_list,
     'numero_divisao': divisao_pedido_list,
     'data_divisao': data_divisao_pedido_list,
     'titulo': titulo_pedido_list,
     'depositante': nome_completo_list,
     'uf': uf_list,
     'sigla': sigla_list,
     'inventor': inventor_nome_completo_list,
     'numero_pct': numero_pct_list,
     'data_pct': data_pct_list,
     'numero_ompi': numero_ompi_list,
     'data_ompi': data_ompi_list,
     'classe_internacional': classe_internacional_list,
     'classe_nacional': classe_nacional_list,
     'prioridade': prioridade_list,
     'sigla_prioridade': sigla_prioridade_list,
     'numero_prioridade': numero_prioridade_list,
     'data_prioridade': data_prioridade_list,
     'prioridade_interna': prioridade_interna_list,
     'comentario': comentario_list,
     'texto_despacho': titulo_list
    })

In [9]:
data_frame["numero_new"] = data_frame["numero"].apply(ConverteNumero)
# https://pt.stackoverflow.com/questions/564421/aplicar-fun%c3%a7%c3%a3o-lamba-para-uma-subtra%c3%a7%c3%a3o-e-soma-python/564425#564425

In [10]:
print(len(data_frame.index))
data_frame[["despacho","numero_new"]]


4189


,despacho,numero_new
0,120,PI0706432
1,120,PI0809125
2,120,PI1101914
3,121,122015028465
4,121,PI0803782
...,...,...
4184,28.30,112021023675
4185,28.30,112022000154
4186,28.30,122020001313
4187,28.40,102016025259


In [11]:
for i, infos in data_frame.iterrows():
    numero = infos.numero_new
    despacho = infos.despacho
    comando = f"INSERT INTO arquivadosxml (id, despacho, numero, data, divisao, anulado, prmexame) VALUES (NULL, '{despacho}', '{numero}', '{data_revista}', '', 0, 0);"
    cursor.execute(comando)
    conexao.commit()

In [15]:
#confere total de registros gravados
comando = f'select count(*) as total from arquivadosxml where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    if len(data_frame.index) == row[0]:
        print("Sucesso na gravação do total de "+str(row[0])+" registros !")
    else:
        print("Total de registros gravados: "+str(row[0])+" registros, quando toral era de "+len(data_frame.index))



Sucesso na gravação do total de 4189 registros !


In [12]:
#confere total de registros gravados com a tabela padrão arquivados
comando = f'select * from arquivados where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    despacho = row[1]
    numero = row[2]
    data = row[3]
    df = data_frame.loc[data_frame['numero_new']==numero]
    if df.empty:
        print("Não encontrei "+str(numero)+" na revista lida")
    else:
        max = len(df)
        ok = False
        for i in range(max):
            id = df.index[i]
            #print("Encontrei "+str(numero)+" na revista lida")
            despacho_rpi=df.loc[id].despacho
            if despacho_rpi in ['100','102','103','104','106','111','112','113','115','116','120','121','130','131','132','133','134','135','136','137','138','139','140','141']:
                despacho_rpi = 'PR - Recursos'
            if despacho_rpi in ['200','201','204','205','206','210','211','212','213','214','215','216','217','218','219','220']:
                despacho_rpi = 'PR - Nulidades'
            if despacho_rpi == despacho:
                ok = True
                
        if (ok == False):
            print("Despacho diferente "+str(numero)+" "+str(despacho)+" "+str(despacho_rpi))
        #print(numero,despacho,despacho_rpi)
    #break
print("Processamento encerrado")

ProgrammingError: 1146 (42S02): Table 'producao.arquivados' doesn't exist

In [591]:
#confere total de registros gravados com a tabela padrão arquivados
comando = f'select * from arquivadosxml where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    despacho = row[1]
    numero = row[2]
    data = row[3]
    if despacho in ['100','102','103','104','106','111','112','113','115','116','120','121','130','131','132','133','134','135','136','137','138','139','140','141']:
        despacho = 'PR - Recursos'
    if despacho in ['200','201','204','205','206','210','211','212','213','214','215','216','217','218','219','220']:
        despacho = 'PR - Nulidades'
    comando = f'select * from arquivados where numero="{numero}" and despacho="{despacho}" and data="{data}"'
    cursor.execute(comando)
    if cursor.fetchone() == None:
        print("Não encontrei "+str(numero)+", "+str(despacho)+" na tabela arquivados")

print("Processamento encerrado")

Processamento encerrado


In [ ]:
df1 = data_frame.loc[data_frame['numero_new']=='112021020724']
df1.head(1)
len(df1)
id = df1.index[0]
print(id)
df1.loc[id]
df1.loc[id].despacho